## Exercício prático de regressão linear

Instruções: Use o dataset 'datasetCarros.csv' em todos os exercícios.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
from scipy import stats

In [45]:
dataset = pd.read_csv('datasetCarros.csv')

In [61]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Nome             301 non-null    object 
 1   Ano              301 non-null    int64  
 2   PrecoVenda       301 non-null    float64
 3   PrecoAtual       301 non-null    float64
 4   KmRodado         301 non-null    int64  
 5   TipoCombustivel  301 non-null    object 
 6   Trasmissao       301 non-null    object 
 7   Owner            301 non-null    int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 18.9+ KB


# 1
a) Faça um modelo de regressão linear simples utilizando a variável 'KmRodado' para prever a 'PrecoVenda'.<br>

b) Calcule o R2 para o modelo criado.

O r^2 estava muito ruim, portanto, decidi tentar tratar alguns outliers. <br>

iniciei calculando o z-score e tirando todas as entradas que tinham z-score > 3

In [57]:
numeric_columns = dataset.select_dtypes(include=['number']).columns
data_numeric = dataset[numeric_columns]

z_scores = stats.zscore(data_numeric)

data_cleaned = dataset[(np.abs(z_scores) < 3).all(axis=1)]

print(data_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Index: 270 entries, 0 to 300
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Nome             270 non-null    object 
 1   Ano              270 non-null    int64  
 2   PrecoVenda       270 non-null    float64
 3   PrecoAtual       270 non-null    float64
 4   KmRodado         270 non-null    int64  
 5   TipoCombustivel  270 non-null    object 
 6   Trasmissao       270 non-null    object 
 7   Owner            270 non-null    int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 19.0+ KB
None


Aqui, ainda estava obtendo resultados muito ruins, então apliquei transformação logarítmica, porém, eu tinha valores que eram 0, então, somei uma pequena constante para poder abordá-los na transformação também.

In [58]:
data_log = data_cleaned.copy()
data_log[numeric_columns] = np.log(data_log[numeric_columns] + 1e-6)

In [59]:
model = LinearRegression()
scaler = RobustScaler()

X = data_log[['KmRodado']]
y = data_log['PrecoVenda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.17700999491191483


# 2
a) Separe o dataset em teste (5%) e treino (95%). Use o método 'train_test_split' do sklearn; configure o parâmetro random_state=10. <br>

b) Treine um modelo de regressão linear múltipla no dataset de treino utilizando todas as variáveis (exceto 'Nome') para prever a 'PrecoVenda' e exiba os coeficientes do modelo.<br>

c) Avalie o modelo encontrado utilizando o dataset de teste. Calcule o R2 e MSE.

In [79]:
model_m = LinearRegression()

X = data_log[['KmRodado', 'Ano', 'PrecoAtual', 'TipoCombustivel', 'Trasmissao', 'Owner']]
y = data_log['PrecoVenda']

X_encoded = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.05, random_state=10)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_m.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [83]:
coef = pd.DataFrame({'Variável': X_encoded.columns, 'Coeficiente': model_m.coef_})
print(coef)
intercept = model_m.intercept_
print("Intercepto:", intercept)

                 Variável   Coeficiente
0                KmRodado -4.665945e-02
1                     Ano  3.273386e-01
2              PrecoAtual  1.853272e+00
3                   Owner  1.110223e-16
4  TipoCombustivel_Diesel  1.657235e-01
5  TipoCombustivel_Petrol  4.713279e-02
6       Trasmissao_Manual  2.568524e-02
Intercepto: 1.5084874649910196


In [84]:
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

R2 Score: 0.9809085886449582
MSE: 0.035096428539332786
